In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.initializers import glorot_uniform
from tensorflow.keras import regularizers

import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import load
from keras.models import load_model
import seaborn as sb
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio

# comment the below line if you not using jupyter notebook
%matplotlib inline

In [2]:
print("Started reading the data for training...")
df_ = pd.read_csv('train.csv')
df = df_.copy()
# df_training = df.iloc[:,:].copy()
print("Finished reading the data")

Started reading the data for training...
Finished reading the data


In [3]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['color_type'])
df['color_type'] = le.transform(df['color_type'])

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc_y1 = OneHotEncoder(handle_unknown='ignore')
enc_y1.fit(df[['breed_category']])
y1 = enc_y1.transform(df[['breed_category']]).toarray()
enc_y2 = OneHotEncoder(handle_unknown='ignore')
enc_y2.fit(df[['pet_category']])
y2 = enc_y2.transform(df[['pet_category']]).toarray()

In [6]:
df[['condition']] = df[['condition']].fillna(3.0)

In [7]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,18,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,53,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,15,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,53,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,2,0.50,11.06,18,4,0.0,1


In [8]:
x = df[['condition', 'color_type', 'length(m)', 'height(cm)', 'X1', 'X2']].values

In [9]:
def scaleData(x, y):
    sc_x = StandardScaler()
    sc_y = StandardScaler()

    sc_x.fit(x)
    sc_y.fit(y)
    
    return sc_x, sc_y

In [10]:
sc_x, sc_y = scaleData(x,y1)

In [11]:
# retrieved train set params

In [12]:
print("Started reading the data for training...")
df_ = pd.read_csv('test.csv')
df = df_.copy()
print("Finished reading the data")

Started reading the data for training...
Finished reading the data


In [13]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [14]:
df['color_type'] = le.transform(df['color_type'])

In [15]:
df[['condition']] = df[['condition']].fillna(3.0)

In [16]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,2,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,38,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,2,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,2,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,15,0.71,30.92,0,7


In [17]:
x = df[['condition', 'color_type', 'length(m)', 'height(cm)', 'X1', 'X2']].values

In [18]:
sc_x, sc_y = scaleData(x,y1)

In [19]:
x_test = sc_x.transform(x)

In [54]:
model = load_model('model_cond-4-64-split_3.h5')

In [55]:
ymodel = model.predict(x_test)

In [56]:
ymodel1 = enc_y1.inverse_transform(ymodel[0])
ymodel2 = enc_y2.inverse_transform(ymodel[1])

In [57]:
ypred = np.append(ymodel1,ymodel2,axis=1)

In [58]:
df_y = pd.DataFrame(ypred)

In [59]:
df_y.head()

,0,1
0,1.0,2.0
1,0.0,1.0
2,0.0,2.0
3,0.0,1.0
4,0.0,2.0


In [60]:
df_out = df[['pet_id']].join(df_y)

In [61]:
df_out.rename(columns={0: 'breed_category', 1: 'pet_category'}, inplace=True)

In [62]:
df_out.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2.0
1,ANSL_76663,0.0,1.0
2,ANSL_58259,0.0,2.0
3,ANSL_67171,0.0,1.0
4,ANSL_72871,0.0,2.0


In [ ]:
df_out.to_csv('output.csv',index=False)